In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../src')
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import scanpy as sc
from spaceoracle import SpaceTravLR
from spaceoracle.tools.network import RegulatoryFactory
from spaceoracle.gene_factory import GeneFactory

/ihome/djishnu/alw399/.local/lib/python3.9/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound
2025-06-22 22:30:43.598869: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /ihome/crc/install/gcc-12.2.0/postgresql/15.2/lib:/ihome/crc/install/gcc-12.2.0/imagemagick/6.9.12/lib:/ihome/crc/install/gcc-12.2.0/geos/3.9.4/lib:/ihome/crc/install/gcc-12.2.0/zstd/zstd-1.5.5/lib:/ihome/crc/install/gcc-12.2.0/unixodbc/2.3.11/lib:/ihome/crc/install/sodium/1.0.18/lib:/ihome/crc/install/gcc-12.2.0/r/4.4.0/lib64/R/lib:/ihome/crc/install/gcc-12.2

In [3]:
sample = 1
base_dir = '/ix/djishnu/shared/djishnu_kor11/'

co_grn = RegulatoryFactory(
    colinks_path=base_dir + f'training_data_2025/mouse_lymph{sample}_visiumHD_colinks.pkl',
    annot='cell_type_int'
)

### Subsample

In [3]:
adata = sc.read_h5ad(
    base_dir + f'training_data_2025/mouse_lymph{sample}_visiumHD.h5ad')

adata


AnnData object with n_obs × n_vars = 89296 × 2116
    obs: 'sample', 'cell_type', 'secondary_annot', 'cell_type_int'
    uns: 'cell_thresholds', 'cell_type_colors', 'neighbors', 'received_ligands', 'received_ligands_tfl', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial', 'spatial_features', 'spatial_maps', 'spatial_unscaled'
    layers: 'imputed_count', 'normalized_count', 'raw_count'
    obsp: 'connectivities', 'distances'

In [4]:
del adata.obsm['spatial_maps']
del adata.obsm['spatial_features']
del adata.uns['received_ligands']
del adata.uns['received_ligands_tfl']


In [5]:
# we need tto split into batches, it's impossible to train this many cells in one go
import numpy as np 

batch_size = adata.n_obs // 4
pool = set(adata.obs_names)

parta = np.random.choice(list(pool), size=min(len(pool), batch_size), replace=False)
parta_set = set(parta)

partb = np.random.choice(list(pool - parta_set), size=min(len(pool - parta_set), batch_size), replace=False)
partb_set = set(partb)

partc = np.random.choice(list(pool - parta_set - partb_set), size=min(len(pool - parta_set - partb_set), batch_size), replace=False)
partc_set = set(partc)

partd_set = pool - parta_set - partb_set - partc_set
partd = list(partd_set)

adata_pta = adata[parta].copy()
adata_ptb = adata[partb].copy()
adata_ptc = adata[partc].copy()
adata_ptd = adata[partd].copy()


In [6]:
used = (
    set(adata_ptd.obs_names)
    | set(adata_ptc.obs_names)
    | set(adata_ptb.obs_names)
    | set(adata_pta.obs_names)
)

assert set(pool) == set(used)

In [7]:
del adata
import gc
gc.collect()

299

In [8]:
from spaceoracle.models.parallel_estimators import init_received_ligands
import numpy as np
from spaceoracle.models.spatial_map import xyc2spatial_fast
from spaceoracle.models.parallel_estimators import create_spatial_features


In [9]:
adata_pta

AnnData object with n_obs × n_vars = 22324 × 2116
    obs: 'sample', 'cell_type', 'secondary_annot', 'cell_type_int'
    uns: 'cell_thresholds', 'cell_type_colors', 'neighbors', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial', 'spatial_unscaled'
    layers: 'imputed_count', 'normalized_count', 'raw_count'
    obsp: 'connectivities', 'distances'

In [10]:
radius=800
contact_distance=50
spatial_dim = 64

for part_name, adata in zip(
                    ['a', 'b', 'c', 'd'], 
                    [adata_pta, adata_ptb, adata_ptc, adata_ptd]):
    
    adata = init_received_ligands(
        adata,
        radius=radius, 
        contact_distance=contact_distance, 
        cell_threshes=adata.uns['cell_thresholds']
    )

    xy = np.array(adata.obsm['spatial'])
    cluster_annot = 'cell_type_int'
    cluster_labels = np.array(adata.obs[cluster_annot])

    adata.obsm['spatial_maps'] = xyc2spatial_fast(
        xyc = np.column_stack([xy, cluster_labels]),
        m=spatial_dim,
        n=spatial_dim,
    )

    adata.obsm['spatial_features'] = create_spatial_features(
        adata.obsm['spatial'][:, 0], 
        adata.obsm['spatial'][:, 1], 
        adata.obs['cell_type_int'], 
        adata.obs.index,
        radius=radius
    )

    adata.write_h5ad(base_dir + f'training_data_2025/mouse_lymph{sample}{part_name}_visiumHD.h5ad')



### Load up

In [5]:
sample = 1
part_name = 'a'
adata = sc.read_h5ad(base_dir + f'training_data_2025/mouse_lymph{sample}{part_name}_visiumHD.h5ad')

In [6]:
from spaceoracle.models.parallel_estimators import SpatialCellularProgramsEstimator

estimator = SpatialCellularProgramsEstimator(
    adata, 
    target_gene='Gata3', 
    cluster_annot='cell_type_int',
    radius=800, 
    contact_distance=50,
    grn=co_grn,
)

In [7]:
sys.getsizeof(adata)

5578730502

In [8]:
%%time
estimator.fit(num_epochs=100, learning_rate=5e-3, 
    score_threshold=0.2, l1_reg=1e-9, estimator='lasso'
)

Fitting Gata3 with 304 modulators
	25 Transcription Factors
	242 Ligand-Receptor Pairs
	37 TranscriptionFactor-Ligand Pairs
0: 0.6709 | 0.6554
1: 0.3105 | 0.2916
2: 0.7170 | 0.7158
3: 0.6567 | 0.5432
4: 0.5931 | 0.5785
5: 0.8914 | 0.8881
6: 0.6506 | 0.6106
7: 0.3412 | 0.3269
8: 0.8612 | 0.8432
CPU times: user 6min 24s, sys: 17min 42s, total: 24min 6s
Wall time: 3min 43s


In [9]:
betas = estimator.get_betas()
betas

,beta0,beta_Atf3,beta_Batf3,beta_Cebpb,beta_Egr1,beta_Fos,beta_Fosb,beta_Hic1,beta_Irf1,beta_Irf4,...,beta_Il15#Smad1,beta_Tnfsf11#Stat2,beta_Csf1#Tcf7l2,beta_Il15#Tcf7l2,beta_Ccl12#Tfap4,beta_Il15#Xbp1,beta_Il2#Xbp1,beta_Il15#Zbtb7a,beta_Clcf1#Zfp281,beta_Il12b#Zfp281
object_id,,,,,,,,,,,,,,,,,,,,,
276144,0.004414,0.005560,0.190928,0.016553,0.089883,0.031401,-0.028544,0.044147,0.007599,0.040965,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
260035,0.002776,0.003059,0.247999,0.027177,0.108382,0.035084,-0.031704,0.058368,0.007774,0.048791,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
177075,0.004705,0.021374,0.182477,0.024351,0.083056,0.024474,-0.026289,0.031496,0.006890,0.031907,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
202871,0.004815,0.014114,0.162284,0.015715,0.082784,0.024149,-0.022953,0.030960,0.007113,0.033600,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278815,0.003490,0.000100,0.271575,0.029219,0.120260,0.035367,-0.031846,0.060546,0.008853,0.051822,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261095,0.004275,0.023604,0.193847,0.016216,0.083735,0.019561,-0.031170,0.016803,0.008626,0.011171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
191458,0.004383,0.021578,0.199370,0.021827,0.079361,0.023623,-0.029597,0.025763,0.007556,0.021509,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
175646,0.004626,0.019210,0.184912,0.025962,0.091485,0.029340,-0.027194,0.038075,0.006551,0.036905,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
lr_terms = [x for x in betas.columns if '$' in x]
betas[lr_terms]

,beta_Tgfb1$Tgfbr1,beta_Tgfb1$Tgfbr2,beta_Tgfb3$Tgfbr1,beta_Tgfb3$Tgfbr2,beta_Gdf11$Tgfbr1,beta_Vegfb$Flt1,beta_Pgf$Flt1,beta_Igf1$Igf1r,beta_Igf1$Itgav,beta_Igf1$Itgb3,...,beta_Sema7a$Plxnc1,beta_Thy1$Itgam,beta_Thy1$Itgb2,beta_Thy1$Itgax,beta_Thy1$Itgav,beta_Thy1$Itgb3,beta_Itga4$Vcam1,beta_Itgb1$Vcam1,beta_Itga9$Vcam1,beta_Itgb7$Vcam1
object_id,,,,,,,,,,,,,,,,,,,,,
276144,-0.000021,-0.000467,0.0,-0.000000,-5.949523e-07,0.0,0.0,0.0,-1.218339e-07,0.0,...,0.0,0.0,-2.403122e-07,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0
260035,-0.000022,-0.000536,0.0,-0.000000,-5.162424e-07,0.0,0.0,0.0,-1.155935e-07,0.0,...,0.0,0.0,-2.700985e-07,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0
177075,-0.000020,-0.000531,0.0,-0.000000,-6.705110e-07,0.0,0.0,0.0,-1.310059e-07,0.0,...,0.0,0.0,-2.054463e-07,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0
202871,-0.000021,-0.000523,0.0,-0.000000,-6.252750e-07,0.0,0.0,0.0,-1.170386e-07,0.0,...,0.0,0.0,-2.223326e-07,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0
278815,-0.000020,-0.000587,0.0,-0.000000,-3.658402e-07,0.0,0.0,0.0,-9.911783e-08,0.0,...,0.0,0.0,-3.076713e-07,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261095,-0.000020,-0.000463,0.0,-0.000000,-7.210921e-07,0.0,0.0,0.0,-1.310453e-07,0.0,...,0.0,0.0,-2.129834e-07,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0
191458,-0.000021,-0.000515,0.0,-0.000000,-6.418174e-07,0.0,0.0,0.0,-1.219176e-07,0.0,...,0.0,0.0,-2.060833e-07,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0
175646,-0.000019,-0.000556,0.0,-0.000000,-6.268695e-07,0.0,0.0,0.0,-1.213508e-07,0.0,...,0.0,0.0,-2.125633e-07,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0


In [11]:
betas[lr_terms].mean(axis=0).sort_values()

beta_Mif$Cd74       -0.003423
beta_Lgals9$Ighm    -0.003347
beta_C3$Cr2         -0.002243
beta_Cxcl13$Cxcr5   -0.000915
beta_Tgfb1$Tgfbr2   -0.000607
                       ...   
beta_Lgals9$Ptprc    0.000289
beta_Nampt$Itgb1     0.000322
beta_Ccl21a$Ccr7     0.001367
beta_Il16$Cd4        0.003691
beta_C3$Itgb2        0.006120
Length: 242, dtype: float32

In [12]:
import pandas as pd

path = '/ix/djishnu/shared/djishnu_kor11/lasso_runs/human_tonsil'
slideseq_betas = pd.read_parquet(path + '/GATA3_betadata.parquet')

lr_terms = [x for x in slideseq_betas.columns if '$' in x]
slideseq_betas[lr_terms].mean(axis=0).sort_values()

beta_MIF$CD74        -0.062170
beta_MIF$CXCR4       -0.047253
beta_IGF1$IGF1R      -0.021451
beta_IL7$IL2RG       -0.014173
beta_LTB$LTBR        -0.010762
                        ...   
beta_IL16$CD4         0.077962
beta_ANGPT2$ITGB1     0.104317
beta_IL7$IL7R         0.108542
beta_ANGPTL1$ITGB1    0.145424
beta_MIF$CD44         0.200008
Length: 804, dtype: float32

In [14]:
star = SpaceTravLR(
    adata=adata,
    annot='cell_type_int', 
    max_epochs=100, 
    learning_rate=5e-3, 
    spatial_dim=64,
    batch_size=512,
    grn=co_grn,
    radius=800,
    contact_distance=50,
    save_dir=base_dir + f'lasso_runs/mouse_lymph{sample}{part_name}_visiumHD',
    scale_factor=1
)

star.run()

Fitting Trps1 with 320 modulators
	22 Transcription Factors
	242 Ligand-Receptor Pairs
	56 TranscriptionFactor-Ligand Pairs
0: 0.2727 | 0.2412
1: x.xxx* | 0.0839
2: 0.5852 | 0.5743
3: 0.7518 | 0.5527
4: x.xxx* | 0.1203
5: 0.7977 | 0.7702
6: 0.7251 | 0.7000
7: 0.2701 | 0.2379
8: 0.5862 | 0.5341
Fitting Ccr4 with 369 modulators
	31 Transcription Factors
	239 Ligand-Receptor Pairs
	99 TranscriptionFactor-Ligand Pairs
0: 0.6503 | 0.6369
1: 0.5775 | 0.5650
2: 0.8470 | 0.8434
3: 0.3787 | 0.5872
4: 0.6990 | 0.6877
5: 0.9270 | 0.9214
6: 0.8154 | 0.7799
7: 0.7745 | 0.7727
8: 0.9323 | 0.9355
Fitting Pms2 with 274 modulators
	9 Transcription Factors
	242 Ligand-Receptor Pairs
	23 TranscriptionFactor-Ligand Pairs
0: x.xxx* | 0.0265
1: x.xxxx | 0.2148
2: 0.2830 | 0.2708
3: 0.3778 | 0.1791
4: x.xxx* | 0.1254
5: 0.4888 | 0.4512
6: 0.5696 | 0.5388
7: x.xxx* | 0.0611
8: 0.6264 | 0.5548
Fitting Ino80b with 318 modulators
	21 Transcription Factors
	242 Ligand-Receptor Pairs
	55 TranscriptionFactor-Ligand

KeyboardInterrupt: 

### Full training

In [4]:
sample = 1
adata = sc.read_h5ad(base_dir + f'training_data_2025/mouse_lymph{sample}_visiumHD.h5ad')
adata

AnnData object with n_obs × n_vars = 89296 × 2116
    obs: 'sample', 'cell_type', 'secondary_annot', 'cell_type_int'
    uns: 'cell_thresholds', 'cell_type_colors', 'neighbors', 'received_ligands', 'received_ligands_tfl', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial', 'spatial_features', 'spatial_maps', 'spatial_unscaled'
    layers: 'imputed_count', 'normalized_count', 'raw_count'
    obsp: 'connectivities', 'distances'

In [7]:
from spaceoracle.models.parallel_estimators import SpatialCellularProgramsEstimator

estimator = SpatialCellularProgramsEstimator(
    adata, 
    target_gene='Ccr4', 
    cluster_annot='cell_type_int',
    radius=600, 
    contact_distance=40,
    grn=co_grn,
)

In [8]:
estimator.fit(num_epochs=100, learning_rate=5e-3, 
    score_threshold=0.2, l1_reg=1e-9, estimator='lasso'
)

Fitting Ccr4 with 360 modulators
	31 Transcription Factors
	230 Ligand-Receptor Pairs
	99 TranscriptionFactor-Ligand Pairs
0: 0.6173 | 0.6012


KeyboardInterrupt: 

In [ ]:
betas = estimator.get_betas()
betas

In [ ]:
betas.to_parquet(f'/ix/djishnu/shared/djishnu_kor11/lasso_runs/mouse_lymph{sample}_visiumHD/Ccr4_betadata.parquet')